In [1]:
#!/usr/bin/env python

# Copyright (c) 2021- The University of Notre Dame.
# This software is distributed under the GNU General Public License.
# See the file COPYING for details.

# This program is a demonstration of using Work Queue with PyTask
# PyTask allows python functions and their arguments to be bottled up and sent as a task to workers
# This example program finds prime numbers in a range
# PyTask sends the function to find the primality of a given number as a task for all numbers in the given range

# How to work this program:
# Run this cell, and afterwards two sliders and a textbox will appear containing parameters for the program that can be adjusted
# Starting number is the first number to be checked for primality, and the Number of tasks is how many numbers after the starting number that will also be checked
# Max number of workers defines a value for what is the largest amount of workers the program will create
# Keep them at the default values for the first time the program is ran
# Afterwards, run the cell below
# Real time output, as well as some informative diagnostic numbers, will then be shown at the very bottom of the notebook
# Grey cells are tasks that have been submitted, yellow cells are tasks that are currently being worked on,
# Dark green cells are tasks that have finished and returned that the number is composite, light green cells are prime numbers
# After the Workqueue Status textbox displays "All tasks complete!", the program is finished! Then feel free to play around with the numbers below

import work_queue as wq
import ipywidgets as widgets
import time
import widget_control 

display = widget_control.Display()
display.create_user_input_widgets()

In [4]:
# function to create the display at the bottom of the notebook
display.create_output_widgets()

def is_prime(number): # this is the function to determine the primality of a number
    import math, time # PyTask uses this function to create the tasks in order to determine which numbers are prime
    for i in range(2, int(math.sqrt(number)) + 1):
            if (number % i == 0):
                return 0 # composite
    return 1 # prime

# create the Work Queue queue and specify the number of workers as well as the properties of those workers
q = wq.WorkQueue(0) # specify the port to create the workers on. 0 selects a random unused port
workers = wq.Factory('local', manager_host_port=f'localhost:{q.port}')
workers.cores = 1
workers.memory = 100
workers.disk = 100
workers.min_workers = 1
workers.max_workers = display.num_workers.value

with workers: # this line instantiates the workers
    # loop through the range of numbers desired and use PyTask to create tasks to determine their primality
    # range from the starting value to the starting value plus the total number of tasks desired
    for i in range(display.tasks_range.value[0], display.tasks_range.value[1] + 1):
        p_task = wq.PythonTask(is_prime, i) # create the task
        p_task.specify_environment("worker-env.tar.gz") # This line ensures that the workers have access to the proper libraries
        q.submit(p_task) # submit the tasks into the queue
        
    while not q.empty(): # continue until all the tasks have been completed
        t = q.wait(5) # give control to Work Queue so that it can talk to workers
        if t:
            if t.return_status != 0: # The task failed
                continue
            display.update_output_widgets(q, t) # update output widgets       
                              